In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import ast
from metrics import *

In [2]:
popularity_df = pd.read_csv("dataset/id_metadata_mmsr.tsv", sep="\t")
popularity_df_spot = pd.read_csv("dataset/id_total_listens.tsv", sep="\t")
popularity_lfm = popularity_df['popularity'].values
infos = pd.read_csv("dataset/id_information_mmsr.tsv", sep="\t")
inter_true = np.loadtxt("./predictions/binary_relevancy_matrix_00.csv", delimiter="\t")
tags = pd.read_csv("./dataset/id_tags_dict.tsv", sep="\t")
genres = pd.read_csv("./dataset/id_genres_mmsr.tsv", sep="\t")

In [3]:
popularity_df_spot.to_numpy()[:, 1:]

array([[1491],
       [2655],
       [21909],
       ...,
       [2283],
       [2043],
       [1980]], shape=(5148, 1), dtype=object)

In [6]:
models = ["random", "tf_idf"]
evaluation = []

for model in tqdm(models):
    inter_pred = np.loadtxt(f"predictions/recs_{model}_10.csv", delimiter="\t")
    pop_at_10 = avg_popularity_at_k(inter_pred, popularity_lfm, 10)
    pop_at_10_spot = avg_popularity_at_k(inter_pred, popularity_df_spot.to_numpy()[:, 1:], 10)
    cov_at_10 = avg_coverage_at_k(inter_pred, inter_true.shape[0], 10)
    ild_at_10 = intra_list_diversity_at_k(inter_pred, inter_true, 10)
    nov_at_10 = novelty_at_k(inter_pred, popularity_lfm, 10)
    div_at_10 = diversity_at_k(inter_pred, tags, genres, 10)

    evaluation.append({
        "model": model,
        "pop@10": pop_at_10,
        "pop_sp@10": pop_at_10_spot,
        "cov@10": cov_at_10,
        "ild@10": ild_at_10,
        "nov@10": nov_at_10,
        "div@10": div_at_10
    })

evaluation_df = pd.DataFrame(evaluation)

100%|██████████| 2/2 [00:32<00:00, 16.42s/it]


In [7]:
evaluation_df

,model,pop@10,pop_sp@10,cov@10,ild@10,nov@10,div@10
0,random,1.001378,1.000718,1.000000,0.937085,-4.962494,29.029332
1,tf_idf,1.030710,0.990188,0.976496,0.904105,-5.003819,28.924437
